In [1]:
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/home/gonzalo/Downloads/ml4cc-general-access_request_pays.json"

In [3]:
from ml4floods.data import utils

aois = utils.read_geojson_from_gcp("gs://ml4cc_data_lake/0_DEV/1_Staging/operational/NEMA001/aois.geojson")
aois.explore()

## Set up the dates to query

In [9]:
import geemap.eefolium as geemap
from datetime import datetime, timezone, timedelta
import ee
from shapely.geometry import box, mapping
from ml4floods.data import ee_download
import geopandas as gpd
import pandas as pd
import folium

date_event = datetime.strptime("2022-09-23","%Y-%m-%d").replace(tzinfo=timezone.utc)

date_start_search = datetime.strptime("2022-09-05","%Y-%m-%d").replace(tzinfo=timezone.utc)
date_end_search = date_start_search + timedelta(days=30)


area_of_interest = aois.geometry.unary_union
print(f"Searching images between {date_start_search} and {date_end_search}")

Searching images between 2022-09-22 00:00:00+00:00 and 2022-10-12 00:00:00+00:00


## Get the `ee.Image` objects to query

In [14]:
%%time 

ee.Initialize()


pol_2_clip = ee.Geometry(mapping(box(*area_of_interest.bounds)))

pol = ee.Geometry(mapping(area_of_interest))

img_col_s2 = ee_download.get_s2_collection(date_start_search, 
                                           date_end_search, pol)

img_col_l8 = ee_download.get_landsat_collection(date_start_search, 
                                                date_end_search, pol)
img_col = img_col_l8.merge(img_col_s2)

def get_info_collection(img_col:ee.ImageCollection)->gpd.GeoDataFrame:
    img_col_info = ee_download.img_collection_to_feature_collection(img_col, 
                                                                ["system:time_start", "valids", "cloud_probability"])
    img_col_info_local = gpd.GeoDataFrame.from_features(img_col_info.getInfo())
    img_col_info_local["valids"]*=100
    img_col_info_local["system:time_start"] = img_col_info_local["system:time_start"].apply(lambda x: datetime.utcfromtimestamp(x/1000))
    return img_col_info_local

img_col_info_l8 = get_info_collection(img_col_l8)
img_col_info_l8["satellite"] = "Landsat"
img_col_info_s2 = get_info_collection(img_col_s2)
img_col_info_s2["satellite"] = "S2"
img_col_info = pd.concat([img_col_info_l8, img_col_info_s2], ignore_index=True)

CPU times: user 763 ms, sys: 6.69 ms, total: 770 ms
Wall time: 55.8 s


In [15]:
img_col_info

,geometry,cloud_probability,system:time_start,valids,satellite
0,"POLYGON ((-180.00000 -90.00000, 180.00000 -90....",12.976994,2022-09-27 23:43:58.015000,10.891085,Landsat
1,"POLYGON ((-180.00000 -90.00000, 180.00000 -90....",77.000371,2022-09-25 23:56:55.781500,30.072944,Landsat
2,"POLYGON ((-180.00000 -90.00000, 180.00000 -90....",0.900647,2022-09-24 00:09:05.508000,19.228646,Landsat
3,"POLYGON ((-180.00000 -90.00000, 180.00000 -90....",0.194682,2022-10-01 00:16:03.260000,0.078338,Landsat
4,"POLYGON ((-180.00000 -90.00000, 180.00000 -90....",93.746478,2022-09-28 23:36:42.210000,0.082406,Landsat
5,"POLYGON ((-180.00000 -90.00000, 180.00000 -90....",39.784663,2022-09-26 23:49:51.550000,27.512145,Landsat
6,"POLYGON ((-180.00000 -90.00000, 180.00000 -90....",37.394552,2022-10-03 23:56:26.069000,35.377826,Landsat
7,"POLYGON ((-180.00000 -90.00000, 180.00000 -90....",0.789949,2022-09-25 00:02:36.775000,40.488477,Landsat
8,"POLYGON ((-180.00000 -90.00000, 180.00000 -90....",78.916835,2022-10-11 00:02:39.670000,40.504972,Landsat
9,"POLYGON ((-180.00000 -90.00000, 180.00000 -90....",1.018367,2022-10-02 00:08:48.013000,19.714810,Landsat


In [25]:
img_col_info_date = img_col_info.sort_values(["system:time_start"])
img_col_info_date

,geometry,cloud_probability,system:time_start,valids,satellite
12,"POLYGON ((-180.00000 -90.00000, 180.00000 -90....",66.478163,2022-09-22 00:26:54.452500,26.476420,S2
13,"POLYGON ((-180.00000 -90.00000, 180.00000 -90....",96.921404,2022-09-22 23:55:00.021000,2.484658,S2
10,"POLYGON ((-180.00000 -90.00000, 180.00000 -90....",98.305371,2022-09-23 00:15:45.815000,0.096712,Landsat
2,"POLYGON ((-180.00000 -90.00000, 180.00000 -90....",0.900647,2022-09-24 00:09:05.508000,19.228646,Landsat
14,"POLYGON ((-180.00000 -90.00000, 180.00000 -90....",33.386913,2022-09-24 00:15:48.975000,61.339829,S2
7,"POLYGON ((-180.00000 -90.00000, 180.00000 -90....",0.789949,2022-09-25 00:02:36.775000,40.488477,Landsat
1,"POLYGON ((-180.00000 -90.00000, 180.00000 -90....",77.000371,2022-09-25 23:56:55.781500,30.072944,Landsat
15,"POLYGON ((-180.00000 -90.00000, 180.00000 -90....",80.601376,2022-09-26 00:05:46.473000,35.128098,S2
5,"POLYGON ((-180.00000 -90.00000, 180.00000 -90....",39.784663,2022-09-26 23:49:51.550000,27.512145,Landsat
16,"POLYGON ((-180.00000 -90.00000, 180.00000 -90....",42.010206,2022-09-27 00:26:47.651500,26.471925,S2


## Show the images with `geemap`

In [22]:
%%time

m = geemap.Map()

imgs_list = img_col.toList(img_col_info.shape[0], 0)
for i in range(img_col_info.shape[0]):
    if img_col_info_local.iloc[i]["cloud_probability"] > 90:
        continue
    
    img_show = ee.Image(imgs_list.get(i))
    
    item = img_col_info.iloc[i]
    bands = ["B11","B8","B4"] if item["satellite"] == "S2" else ["B6","B5","B4"]
    m.addLayer(img_show.clip(pol), 
                 {"min":0, "max":3000, 
                  "bands":bands},
                 f"({i}/{img_col_info.shape[0]}) {item['satellite']} SWIR/NIR/RED {item['system:time_start'].strftime('%Y-%m-%d')}", 
                 False)


m.addLayer(pol, {"color": 'FF000000'}, "AoI")

m.centerObject(pol, zoom=9)

In [24]:
folium.LayerControl(collapsed=False).add_to(m)
m